# Проведем оценку работы нескольких моделей и выберем лучшую

Проверим следующие модели:
- dummy model (mean by brand_name)
- линейные модели (с регуляризацией)
- случайный лес
- cat boost

Также для передачи в модель данные требуется векторизовать


In [1]:
import pandas as pd
import numpy as np
from pprint import pprint

from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.base import BaseEstimator

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge, LinearRegression, RidgeCV
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor, Pool

from ngboost import NGBRegressor
from ngboost.distns import Normal
from scipy.stats import norm

from sklearn.metrics import mean_squared_error


In [2]:
SEED = 42

# Прочитаем данные

In [3]:
df = pd.read_csv("../data/processed/clean_data.csv")

In [4]:
df.head()

,brand_name,priceLog,proc_freq,proc_brand,proc_name,proc_count,videocard,videocard_memory,screen,ssd_volume,ram,hdmi,material,battery_life
0,other,11.156251,1.600000,intel,intel core i5,4.0,интегрированная,0.0,15.0,512,16,True,металл,7.0
1,other,11.561716,2.000000,intel,intel core i5,8.0,geforce rtx,4.0,17.0,512,16,True,пластик,5.0
2,other,11.002100,1.600000,intel,intel core i5,4.0,интегрированная,0.0,15.0,512,8,True,металл,7.0
3,apple,11.849398,3.406613,apple,apple m2,8.0,интегрированная,0.0,15.0,256,8,False,металл,20.0
4,msi,11.775290,2.700000,intel,intel core i5,6.0,geforce rtx,6.0,17.0,512,16,True,металл,5.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 933 entries, 0 to 932
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   brand_name        933 non-null    object 
 1   priceLog          933 non-null    float64
 2   proc_freq         933 non-null    float64
 3   proc_brand        933 non-null    object 
 4   proc_name         933 non-null    object 
 5   proc_count        933 non-null    float64
 6   videocard         933 non-null    object 
 7   videocard_memory  933 non-null    float64
 8   screen            933 non-null    float64
 9   ssd_volume        933 non-null    int64  
 10  ram               933 non-null    int64  
 11  hdmi              933 non-null    bool   
 12  material          933 non-null    object 
 13  battery_life      933 non-null    float64
dtypes: bool(1), float64(6), int64(2), object(5)
memory usage: 95.8+ KB


In [6]:
df.sample(1).to_dict(orient="records")

[{'brand_name': 'lenovo',
  'priceLog': 11.554067408301403,
  'proc_freq': 3.1,
  'proc_brand': 'intel',
  'proc_name': 'intel core i5',
  'proc_count': 4.0,
  'videocard': 'geforce rtx',
  'videocard_memory': 4.0,
  'screen': 15.0,
  'ssd_volume': 512,
  'ram': 8,
  'hdmi': True,
  'material': 'пластик',
  'battery_life': 5.0}]

# Разделим на трейн и тест

In [7]:
df_train, df_test = train_test_split(df, shuffle=True, random_state=SEED)

In [8]:
y_train = df_train.priceLog
y_test = df_test.priceLog
X_train = df_train.drop("priceLog", axis=1)
X_test = df_test.drop("priceLog", axis=1)

# Простая модель

In [9]:
class DummyMeanregressor(BaseEstimator):
    """
    Берет среднюю цены по бренду
    """
    def __init__(self):
        self._fitted = False
        
    def fit(self, X, y):
        X_ = X.copy()
        X_["target"] = y
        self._data = X_.groupby("brand_name").target.mean()
        self._fitted = True
    
    def predict(self, X):
        if not self._fitted:
            raise "Model not trained"
        return self._data[X["brand_name"]]
    
    def score(self, X, y):
        return mean_squared_error(self.predict(X), y)
    

In [10]:
def score_model(model, X, y):
    result = list(np.round((-cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error'))**0.5, 5))
    print(f"Cross validation RMSE for {model}")
    print()
    pprint(result)
    print()
    print(f"Mean RMSE: {np.round(np.mean(result), 5)}")
   

In [11]:
score_model(DummyMeanregressor(), X_train, y_train)

Cross validation RMSE for DummyMeanregressor()

[0.56665, 0.64666, 0.57335, 0.60871, 0.60959]

Mean RMSE: 0.60099


# Линейная модель

Для линейной модели сделаем шкалирование признаков и OHE категориальных признаков

### Шкалирование

In [12]:
X_train_linear = X_train.copy()
X_test_linear = X_test.copy()

In [13]:
numerical = X_train_linear.select_dtypes(include=[int, float]).columns.tolist()

In [14]:
scaler = StandardScaler()

In [15]:
X_train_linear[numerical] = scaler.fit_transform(X_train_linear[numerical])

In [16]:
X_test_linear[numerical] = scaler.transform(X_test_linear[numerical])

### Преобразуем категориальные фичи

In [17]:
dv = DictVectorizer(sparse=False)

In [18]:
X_train_linear = dv.fit_transform(X_train_linear.to_dict(orient="records"))
X_test_linear = dv.transform(X_test_linear.to_dict(orient="records"))

### Смотрим результаты

In [19]:
score_model(LinearRegression(), X_train_linear, y_train)

Cross validation RMSE for LinearRegression()

[0.24531, 0.30914, 0.27929, 0.30839, 0.26309]

Mean RMSE: 0.28104


In [20]:
regr_cv = RidgeCV(alphas=[0.1, 1.0, 10.0])

In [21]:
model_cv = regr_cv.fit(X_train_linear, y_train)

In [22]:
print(f"Param alpha = {model_cv.alpha_}")

Param alpha = 1.0


In [23]:
score_model(model_cv, X_train_linear, y_train)

Cross validation RMSE for RidgeCV(alphas=[0.1, 1.0, 10.0])

[0.24495, 0.30818, 0.28006, 0.31012, 0.24817]

Mean RMSE: 0.2783


> У ленейных моделей результаты оказались лучше чем у простой модели, основанных на средних ценах
> Сложно оценить погрешность, в реальных значениях, т.к. мы использовали логарифм цены.
> Но видно существенное улучшение
>
> Также заметим что модель с регуляризацией показала еще более лучший результат (но не значительно)

# Случайный лес

Также требует преобразования категориальных данных, но шкалирование делать уже не нужно

In [24]:
X_train_forest = X_train.copy()
X_test_forest = X_test.copy()

In [25]:
dv = DictVectorizer(sparse=False)
X_train_forest = dv.fit_transform(X_train_forest.to_dict(orient="records"))
X_test_forest = dv.transform(X_test_forest.to_dict(orient="records"))

### Смотрим результаты

In [26]:
for depth in [1,3,7,10, 12, 15]:
    print(f"DEPTH {depth}")
    score_model(RandomForestRegressor(n_estimators=200, max_depth=depth, random_state=SEED), X_train_forest, y_train)


DEPTH 1
Cross validation RMSE for RandomForestRegressor(max_depth=1, n_estimators=200, random_state=42)

[0.42047, 0.52617, 0.4689, 0.51254, 0.46665]

Mean RMSE: 0.47895
DEPTH 3
Cross validation RMSE for RandomForestRegressor(max_depth=3, n_estimators=200, random_state=42)

[0.28884, 0.31856, 0.30888, 0.29269, 0.28551]

Mean RMSE: 0.2989
DEPTH 7
Cross validation RMSE for RandomForestRegressor(max_depth=7, n_estimators=200, random_state=42)

[0.22894, 0.257, 0.24301, 0.19895, 0.21418]

Mean RMSE: 0.22842
DEPTH 10
Cross validation RMSE for RandomForestRegressor(max_depth=10, n_estimators=200, random_state=42)

[0.2261, 0.24968, 0.24514, 0.19129, 0.20566]

Mean RMSE: 0.22357
DEPTH 12
Cross validation RMSE for RandomForestRegressor(max_depth=12, n_estimators=200, random_state=42)

[0.22676, 0.25008, 0.24494, 0.18977, 0.20511]

Mean RMSE: 0.22333
DEPTH 15
Cross validation RMSE for RandomForestRegressor(max_depth=15, n_estimators=200, random_state=42)

[0.22691, 0.24982, 0.2445, 0.18992, 0.2

In [27]:
modelForest = RandomForestRegressor(random_state=SEED, n_estimators=2000)

In [28]:
modelForest.fit(X_train_forest, y_train)

RandomForestRegressor(n_estimators=2000, random_state=42)

In [29]:
mean_squared_error(modelForest.predict(X_test_forest), y_test)**0.5

0.20272778402077868

> Результаты случайного леса оказались еще лучше  
> Оптимальной глубиной для леса оказалась глубина 10 

# CatBoost

Для catboost не будем делать преобразование, но положим все данные в catboost.Pool, указав какие фичи являются категориальными.

In [30]:
cat_feats = list(X_train.select_dtypes('object').columns)
train_dataset = Pool(X_train, y_train, cat_features=cat_feats) 
test_dataset = Pool(X_test, y_test, cat_features=cat_feats) 

C:\Soft\Anaconda\lib\site-packages\catboost\core.py:1135: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, pairs, weight, group_id, group_weight, subgroup_id, pairs_weight, baseline, feature_names, thread_count)


In [31]:
model = CatBoostRegressor(loss_function='RMSE', random_seed=SEED)

In [32]:
grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}
params, cv_results = model.grid_search(grid, train_dataset, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 10.9410282	test: 10.7834270	best: 10.7834270 (0)	total: 203ms	remaining: 20.1s
1:	learn: 10.6139691	test: 10.4582013	best: 10.4582013 (1)	total: 303ms	remaining: 14.8s
2:	learn: 10.2973418	test: 10.1420704	best: 10.1420704 (2)	total: 423ms	remaining: 13.7s
3:	learn: 9.9899644	test: 9.8356157	best: 9.8356157 (3)	total: 566ms	remaining: 13.6s
4:	learn: 9.6916602	test: 9.5391264	best: 9.5391264 (4)	total: 660ms	remaining: 12.5s
5:	learn: 9.4022637	test: 9.2510682	best: 9.2510682 (5)	total: 761ms	remaining: 11.9s
6:	learn: 9.1216199	test: 8.9726118	best: 8.9726118 (6)	total: 946ms	remaining: 12.6s
7:	learn: 8.8492459	test: 8.7024968	best: 8.7024968 (7)	total: 1.11s	remaining: 12.8s
8:	learn: 8.5851038	test: 8.4394737	best: 8.4394737 (8)	total: 1.25s	remaining: 12.6s
9:	learn: 8.3292910	test: 8.1852663	best: 8.1852663 (9)	total: 1.36s	remaining: 12.3s
10:	learn: 8.0806566	test: 7.9376785	best: 7.9376785 (10)	total: 1.5s	remaining: 12.2s
11:	learn: 7.8396314	test: 7.6984160	best: 7

95:	learn: 0.7085883	test: 0.6419232	best: 0.6419232 (95)	total: 14s	remaining: 584ms
96:	learn: 0.6919585	test: 0.6258984	best: 0.6258984 (96)	total: 14.2s	remaining: 439ms
97:	learn: 0.6756197	test: 0.6104703	best: 0.6104703 (97)	total: 14.3s	remaining: 292ms
98:	learn: 0.6607272	test: 0.5962640	best: 0.5962640 (98)	total: 14.4s	remaining: 146ms
99:	learn: 0.6460998	test: 0.5823599	best: 0.5823599 (99)	total: 14.5s	remaining: 0us

bestTest = 0.5823599285
bestIteration = 99

0:	loss: 0.5823599	best: 0.5823599 (0)	total: 14.9s	remaining: 23m 36s
0:	learn: 10.1556497	test: 10.0000829	best: 10.0000829 (0)	total: 51.2ms	remaining: 5.07s
1:	learn: 9.1441606	test: 8.9946178	best: 8.9946178 (1)	total: 235ms	remaining: 11.5s
2:	learn: 8.2360583	test: 8.0879882	best: 8.0879882 (2)	total: 294ms	remaining: 9.5s
3:	learn: 7.4181245	test: 7.2728859	best: 7.2728859 (3)	total: 427ms	remaining: 10.2s
4:	learn: 6.6812215	test: 6.5416850	best: 6.5416850 (4)	total: 482ms	remaining: 9.16s
5:	learn: 6.018

88:	learn: 0.2427264	test: 0.2205846	best: 0.2205846 (88)	total: 11.6s	remaining: 1.43s
89:	learn: 0.2414161	test: 0.2198690	best: 0.2198690 (89)	total: 11.7s	remaining: 1.3s
90:	learn: 0.2412201	test: 0.2194823	best: 0.2194823 (90)	total: 11.8s	remaining: 1.17s
91:	learn: 0.2408074	test: 0.2191674	best: 0.2191674 (91)	total: 12s	remaining: 1.04s
92:	learn: 0.2407181	test: 0.2191834	best: 0.2191674 (91)	total: 12.1s	remaining: 912ms
93:	learn: 0.2401907	test: 0.2193424	best: 0.2191674 (91)	total: 12.2s	remaining: 782ms
94:	learn: 0.2396153	test: 0.2191435	best: 0.2191435 (94)	total: 12.4s	remaining: 650ms
95:	learn: 0.2385348	test: 0.2185731	best: 0.2185731 (95)	total: 12.5s	remaining: 520ms
96:	learn: 0.2382311	test: 0.2177286	best: 0.2177286 (96)	total: 12.6s	remaining: 391ms
97:	learn: 0.2374391	test: 0.2165349	best: 0.2165349 (97)	total: 12.7s	remaining: 260ms
98:	learn: 0.2371416	test: 0.2163213	best: 0.2163213 (98)	total: 12.9s	remaining: 130ms
99:	learn: 0.2368646	test: 0.216310

81:	learn: 1.0129570	test: 0.9385864	best: 0.9385864 (81)	total: 12.8s	remaining: 2.81s
82:	learn: 0.9861840	test: 0.9123359	best: 0.9123359 (82)	total: 12.9s	remaining: 2.65s
83:	learn: 0.9611220	test: 0.8874097	best: 0.8874097 (83)	total: 13.1s	remaining: 2.5s
84:	learn: 0.9364581	test: 0.8631683	best: 0.8631683 (84)	total: 13.3s	remaining: 2.34s
85:	learn: 0.9120920	test: 0.8396201	best: 0.8396201 (85)	total: 13.4s	remaining: 2.19s
86:	learn: 0.8887546	test: 0.8168329	best: 0.8168329 (86)	total: 13.6s	remaining: 2.03s
87:	learn: 0.8664699	test: 0.7955547	best: 0.7955547 (87)	total: 13.8s	remaining: 1.89s
88:	learn: 0.8445545	test: 0.7742565	best: 0.7742565 (88)	total: 14s	remaining: 1.73s
89:	learn: 0.8236217	test: 0.7540212	best: 0.7540212 (89)	total: 14.2s	remaining: 1.57s
90:	learn: 0.8035323	test: 0.7349129	best: 0.7349129 (90)	total: 14.4s	remaining: 1.42s
91:	learn: 0.7840866	test: 0.7164651	best: 0.7164651 (91)	total: 14.6s	remaining: 1.26s
92:	learn: 0.7655289	test: 0.698216

...

107:	learn: 0.6965521	test: 0.6699809	best: 0.6699809 (107)	total: 37s	remaining: 31.6s
108:	learn: 0.6845875	test: 0.6590072	best: 0.6590072 (108)	total: 37.6s	remaining: 31.4s
109:	learn: 0.6718749	test: 0.6469635	best: 0.6469635 (109)	total: 38.1s	remaining: 31.2s
110:	learn: 0.6626154	test: 0.6385611	best: 0.6385611 (110)	total: 38.5s	remaining: 30.9s
111:	learn: 0.6513730	test: 0.6287493	best: 0.6287493 (111)	total: 39s	remaining: 30.6s
112:	learn: 0.6401359	test: 0.6192852	best: 0.6192852 (112)	total: 39.4s	remaining: 30.3s
113:	learn: 0.6290560	test: 0.6089274	best: 0.6089274 (113)	total: 39.6s	remaining: 29.9s
114:	learn: 0.6187761	test: 0.6005610	best: 0.6005610 (114)	total: 40.1s	remaining: 29.6s
115:	learn: 0.6094170	test: 0.5926170	best: 0.5926170 (115)	total: 40.6s	remaining: 29.4s
116:	learn: 0.6000081	test: 0.5842405	best: 0.5842405 (116)	total: 40.8s	remaining: 29s
117:	learn: 0.5912942	test: 0.5771116	best: 0.5771116 (117)	total: 41.3s	remaining: 28.7s
118:	learn: 0.58

199:	learn: 0.3141595	test: 0.3701819	best: 0.3701819 (199)	total: 1m 18s	remaining: 0us

bestTest = 0.3701819453
bestIteration = 199

94:	loss: 0.3701819	best: 0.1807139 (33)	total: 1h 13m 40s	remaining: 46.5s
0:	learn: 10.3053821	test: 10.1845222	best: 10.1845222 (0)	total: 463ms	remaining: 1m 32s
1:	learn: 9.3509562	test: 9.2493849	best: 9.2493849 (1)	total: 839ms	remaining: 1m 23s
2:	learn: 8.5210868	test: 8.4495793	best: 8.4495793 (2)	total: 1.28s	remaining: 1m 24s
3:	learn: 7.6891050	test: 7.6273265	best: 7.6273265 (3)	total: 1.45s	remaining: 1m 11s
4:	learn: 6.9360859	test: 6.8726223	best: 6.8726223 (4)	total: 1.66s	remaining: 1m 4s
5:	learn: 6.2773099	test: 6.2176359	best: 6.2176359 (5)	total: 1.94s	remaining: 1m 2s
6:	learn: 5.7290349	test: 5.6805731	best: 5.6805731 (6)	total: 2.4s	remaining: 1m 6s
7:	learn: 5.2309078	test: 5.1678762	best: 5.1678762 (7)	total: 2.87s	remaining: 1m 8s
8:	learn: 4.7661804	test: 4.7012413	best: 4.7012413 (8)	total: 3.35s	remaining: 1m 11s
9:	learn

92:	learn: 0.2642549	test: 0.3979529	best: 0.3979529 (92)	total: 39.5s	remaining: 45.5s
93:	learn: 0.2623013	test: 0.3951972	best: 0.3951972 (93)	total: 40s	remaining: 45.1s
94:	learn: 0.2609027	test: 0.3933219	best: 0.3933219 (94)	total: 40.5s	remaining: 44.7s
95:	learn: 0.2588910	test: 0.3923793	best: 0.3923793 (95)	total: 40.9s	remaining: 44.3s
96:	learn: 0.2565260	test: 0.3921366	best: 0.3921366 (96)	total: 41.4s	remaining: 44s
97:	learn: 0.2554209	test: 0.3915771	best: 0.3915771 (97)	total: 42s	remaining: 43.7s
98:	learn: 0.2534427	test: 0.3903087	best: 0.3903087 (98)	total: 42.5s	remaining: 43.3s
99:	learn: 0.2521702	test: 0.3888194	best: 0.3888194 (99)	total: 42.9s	remaining: 42.9s
100:	learn: 0.2504723	test: 0.3867431	best: 0.3867431 (100)	total: 43.4s	remaining: 42.6s
101:	learn: 0.2497195	test: 0.3859599	best: 0.3859599 (101)	total: 43.9s	remaining: 42.2s
102:	learn: 0.2481115	test: 0.3848162	best: 0.3848162 (102)	total: 44.3s	remaining: 41.8s
103:	learn: 0.2475327	test: 0.38

184:	learn: 0.1549400	test: 0.3343913	best: 0.3343913 (184)	total: 1m 25s	remaining: 6.89s
185:	learn: 0.1546444	test: 0.3337939	best: 0.3337939 (185)	total: 1m 25s	remaining: 6.43s
186:	learn: 0.1537624	test: 0.3336898	best: 0.3336898 (186)	total: 1m 25s	remaining: 5.97s
187:	learn: 0.1532795	test: 0.3336154	best: 0.3336154 (187)	total: 1m 26s	remaining: 5.52s
188:	learn: 0.1527487	test: 0.3335070	best: 0.3335070 (188)	total: 1m 26s	remaining: 5.06s
189:	learn: 0.1523130	test: 0.3333197	best: 0.3333197 (189)	total: 1m 27s	remaining: 4.6s
190:	learn: 0.1506537	test: 0.3338641	best: 0.3333197 (189)	total: 1m 27s	remaining: 4.14s
191:	learn: 0.1500087	test: 0.3335709	best: 0.3333197 (189)	total: 1m 28s	remaining: 3.69s
192:	learn: 0.1492273	test: 0.3333877	best: 0.3333197 (189)	total: 1m 29s	remaining: 3.23s
193:	learn: 0.1488295	test: 0.3332410	best: 0.3332410 (193)	total: 1m 29s	remaining: 2.77s
194:	learn: 0.1485298	test: 0.3332342	best: 0.3332342 (194)	total: 1m 30s	remaining: 2.31s


90:	learn: 0.1989193	test: 0.2540896	best: 0.2540896 (90)
91:	learn: 0.1982325	test: 0.2534427	best: 0.2534427 (91)	total: 1m 9s	remaining: 43.8s
92:	learn: 0.1971647	test: 0.2531405	best: 0.2531405 (92)	total: 1m 10s	remaining: 43.1s
93:	learn: 0.1966937	test: 0.2531493	best: 0.2531405 (92)	total: 1m 11s	remaining: 42.4s
94:	learn: 0.1960914	test: 0.2529509	best: 0.2529509 (94)	total: 1m 11s	remaining: 41.7s
95:	learn: 0.1958412	test: 0.2526965	best: 0.2526965 (95)	total: 1m 12s	remaining: 41s
96:	learn: 0.1954326	test: 0.2526470	best: 0.2526470 (96)
97:	learn: 0.1948922	test: 0.2525616	best: 0.2525616 (97)	total: 1m 14s	remaining: 39.5s
98:	learn: 0.1941031	test: 0.2523278	best: 0.2523278 (98)	total: 1m 15s	remaining: 38.7s
99:	learn: 0.1933922	test: 0.2519286	best: 0.2519286 (99)
100:	learn: 0.1929506	test: 0.2519663	best: 0.2519286 (99)	total: 1m 16s	remaining: 37.2s
101:	learn: 0.1924394	test: 0.2519504	best: 0.2519286 (99)	total: 1m 17s	remaining: 36.4s
102:	learn: 0.1916873	test

In [39]:
model._get_params()

{'random_seed': 42,
 'depth': 4,
 'verbose': 0,
 'iterations': 150,
 'l2_leaf_reg': 0.2,
 'learning_rate': 0.1,
 'loss_function': 'RMSE'}

In [40]:
model.save_model("../models/cagboost_model.bin")

# NGBoost

Попробуем модель из вероятностного программирования [NGBoost](https://github.com/stanfordmlgroup/ngboost)  
Она может дать нам не только прогноз, но и доверительные интервалы.

In [34]:
score_model(NGBRegressor(Dist=Normal, random_state=SEED), X_train_forest, y_train)

[iter 0] loss=1.1594 val_loss=0.0000 scale=1.0000 norm=0.8395
[iter 100] loss=0.4862 val_loss=0.0000 scale=2.0000 norm=1.0239
[iter 200] loss=-0.0950 val_loss=0.0000 scale=2.0000 norm=0.8867
[iter 300] loss=-0.3472 val_loss=0.0000 scale=2.0000 norm=0.9048
[iter 400] loss=-0.4496 val_loss=0.0000 scale=2.0000 norm=0.9209
[iter 0] loss=1.1222 val_loss=0.0000 scale=1.0000 norm=0.8207
[iter 100] loss=0.3836 val_loss=0.0000 scale=2.0000 norm=0.9872
[iter 200] loss=-0.1558 val_loss=0.0000 scale=2.0000 norm=0.8877
[iter 300] loss=-0.3747 val_loss=0.0000 scale=2.0000 norm=0.9367
[iter 400] loss=-0.4748 val_loss=0.0000 scale=2.0000 norm=0.9584
[iter 0] loss=1.1347 val_loss=0.0000 scale=1.0000 norm=0.8284
[iter 100] loss=0.4348 val_loss=0.0000 scale=2.0000 norm=1.0125
[iter 200] loss=-0.1407 val_loss=0.0000 scale=2.0000 norm=0.8982
[iter 300] loss=-0.3873 val_loss=0.0000 scale=2.0000 norm=0.9282
[iter 400] loss=-0.4952 val_loss=0.0000 scale=1.0000 norm=0.4752
[iter 0] loss=1.1089 val_loss=0.0000 

In [28]:
modelNG = NGBRegressor(Dist=Normal, random_state=SEED, n_estimators=2000)

In [29]:
modelNG.fit(X_train_forest, y_train)

[iter 0] loss=1.1340 val_loss=0.0000 scale=1.0000 norm=0.8280
[iter 100] loss=0.4223 val_loss=0.0000 scale=2.0000 norm=1.0061
[iter 200] loss=-0.1222 val_loss=0.0000 scale=2.0000 norm=0.8992
[iter 300] loss=-0.3541 val_loss=0.0000 scale=1.0000 norm=0.4668
[iter 400] loss=-0.4572 val_loss=0.0000 scale=1.0000 norm=0.4784
[iter 500] loss=-0.5289 val_loss=0.0000 scale=2.0000 norm=0.9543
[iter 600] loss=-0.5722 val_loss=0.0000 scale=1.0000 norm=0.4691
[iter 700] loss=-0.6053 val_loss=0.0000 scale=1.0000 norm=0.4628
[iter 800] loss=-0.6304 val_loss=0.0000 scale=0.5000 norm=0.2302
[iter 900] loss=-0.6414 val_loss=0.0000 scale=0.2500 norm=0.1143
[iter 1000] loss=-0.6519 val_loss=0.0000 scale=0.0020 norm=0.0009
[iter 1100] loss=-0.6528 val_loss=0.0000 scale=0.0020 norm=0.0009
[iter 1200] loss=-0.6551 val_loss=0.0000 scale=0.5000 norm=0.2286
[iter 1300] loss=-0.6606 val_loss=0.0000 scale=0.0010 norm=0.0004
[iter 1400] loss=-0.6657 val_loss=0.0000 scale=0.0010 norm=0.0004
[iter 1500] loss=-0.6666

NGBRegressor(n_estimators=2000,
             random_state=RandomState(MT19937) at 0x1F06FD32D40)

In [30]:
mean_squared_error(modelNG.predict(X_test_forest), y_test)**0.5

0.20559921257563266

> Модель NG boost рабоает хуже catboost и чуть-чуть хуже случайного леса  
> Но у нее есть возможность предсказывать доверительные интевалы, и давайте воспользуемся этой возможностью

In [60]:
preds = modelNG.pred_param(X_test_forest)

In [103]:
def get_ci(preds, ci_level=0.95):
    lower_bound = (1-ci_level)/2
    z_score = norm.ppf(lower_bound)
    print(z_score)
    
    mean = preds[:,0]
    lower = preds[:,0] + z_score * np.exp(preds[:,1])
    upper = preds[:,0] - z_score * np.exp(preds[:,1])
    
    return np.c_[mean, lower, upper]
    

In [77]:
preds_with_ci = get_ci(preds)
plot_data = np.round(np.expm1(np.c_[y_test, preds_with_ci]), -2)
result_ngboost = pd.DataFrame(plot_data, columns=["y", "pred", "lower95", "upper95"])

In [78]:
result_ngboost

,y,pred,lower95,upper95
0,160000.0,126200.0,93900.0,169700.0
1,50000.0,40500.0,30500.0,53600.0
2,170000.0,140400.0,104600.0,188600.0
3,30000.0,30000.0,21700.0,41600.0
4,300000.0,258500.0,214800.0,311000.0
...,...,...,...,...
229,185000.0,196800.0,149000.0,259900.0
230,60000.0,64200.0,48000.0,85900.0
231,120000.0,94700.0,68800.0,130300.0
232,75000.0,94100.0,69700.0,127100.0


In [86]:
y_in_CI_rate = np.mean((result_ngboost.y > result_ngboost.lower95) & (result_ngboost.y < result_ngboost.upper95))

In [91]:
print(f"Значение y попадает в 95% доверительные интервал в {y_in_CI_rate*100:.0f}%")

Значение y попадает в 95% доверительные интервал в 89%


> Мы видем, что даже доверительные интервалы предсказаны неплохо, и target в них хорошо попадает  
> Наверное это неплохо

# Настройка гиперпараметров для NGBoost
Из доступных для анализа гиперпараметров
- Глубина деревьев базовых алгоритмов
- Количество итераций
- learning rate

## Подберем глубину деревьев

In [28]:
for depth in [1,2,3,5,7]:
    print(f"depth: {depth}")
    score_model(
        NGBRegressor(
            Dist=Normal, random_state=SEED, verbose=False, Base=DecisionTreeRegressor(criterion='friedman_mse', max_depth=depth)), 
        X_train_forest, y_train
    )
    print()
    print()

depth: 1
Cross validation RMSE for NGBRegressor(Base=DecisionTreeRegressor(criterion='friedman_mse', max_depth=1),
             random_state=RandomState(MT19937) at 0x2259022D740, verbose=False)

[0.2632, 0.32227, 0.29772, 0.28085, 0.28636]

Mean RMSE: 0.29008


depth: 2
Cross validation RMSE for NGBRegressor(Base=DecisionTreeRegressor(criterion='friedman_mse', max_depth=2),
             random_state=RandomState(MT19937) at 0x2259022D740, verbose=False)

[0.21943, 0.28723, 0.24262, 0.21161, 0.23717]

Mean RMSE: 0.23961


depth: 3
Cross validation RMSE for NGBRegressor(random_state=RandomState(MT19937) at 0x2259022D740, verbose=False)

[0.21634, 0.26041, 0.23128, 0.19796, 0.22548]

Mean RMSE: 0.22629


depth: 5
Cross validation RMSE for NGBRegressor(Base=DecisionTreeRegressor(criterion='friedman_mse', max_depth=5),
             random_state=RandomState(MT19937) at 0x2259022D740, verbose=False)

[0.22639, 0.25564, 0.24993, 0.20754, 0.2057]

Mean RMSE: 0.22904


depth: 7
Cross validation 

## Подбираем learning_rate

In [31]:
for l_rate in [0.0001, 0.001, 0.01, 0.1]:
    print(f"learning_rate: {l_rate}")
    score_model(
        NGBRegressor(
            Dist=Normal, 
            random_state=SEED, 
            verbose=False, 
            Base=DecisionTreeRegressor(criterion='friedman_mse', max_depth=3),
            learning_rate=l_rate
        ), 
        X_train_forest, y_train
    )
    print()
    print()

learning_rate: 0.0001
Cross validation RMSE for NGBRegressor(learning_rate=0.0001,
             random_state=RandomState(MT19937) at 0x2259022D840, verbose=False)

[0.64413, 0.75815, 0.72076, 0.78955, 0.69953]

Mean RMSE: 0.72242


learning_rate: 0.001
Cross validation RMSE for NGBRegressor(learning_rate=0.001,
             random_state=RandomState(MT19937) at 0x2259022D840, verbose=False)

[0.4634, 0.55001, 0.51228, 0.55264, 0.50597]

Mean RMSE: 0.51686


learning_rate: 0.01
Cross validation RMSE for NGBRegressor(random_state=RandomState(MT19937) at 0x2259022D840, verbose=False)

[0.21605, 0.26108, 0.23104, 0.19818, 0.22483]

Mean RMSE: 0.22624


learning_rate: 0.1
Cross validation RMSE for NGBRegressor(learning_rate=0.1,
             random_state=RandomState(MT19937) at 0x2259022D840, verbose=False)

[0.22064, 0.24845, 0.23546, 0.19279, 0.21473]

Mean RMSE: 0.22241




In [37]:
for l_rate in [0.03, 0.05, 0.07, 0.1, 0.12]:
    print(f"learning_rate: {l_rate}")
    score_model(
        NGBRegressor(
            Dist=Normal, 
            random_state=SEED, 
            verbose=False, 
            Base=DecisionTreeRegressor(criterion='friedman_mse', max_depth=3),
            learning_rate=l_rate,
            early_stopping_rounds=100
        ), 
        X_train_forest, y_train
    )
    print()
    print()

learning_rate: 0.03
Cross validation RMSE for NGBRegressor(early_stopping_rounds=100, learning_rate=0.03,
             random_state=RandomState(MT19937) at 0x2259022D440, verbose=False)

[0.21557, 0.25326, 0.22289, 0.20263, 0.22699]

Mean RMSE: 0.22427


learning_rate: 0.05
Cross validation RMSE for NGBRegressor(early_stopping_rounds=100, learning_rate=0.05,
             random_state=RandomState(MT19937) at 0x2259022D440, verbose=False)

[0.21419, 0.24814, 0.21446, 0.19849, 0.22554]

Mean RMSE: 0.22016


learning_rate: 0.07
Cross validation RMSE for NGBRegressor(early_stopping_rounds=100, learning_rate=0.07,
             random_state=RandomState(MT19937) at 0x2259022D440, verbose=False)

[0.20934, 0.24765, 0.21646, 0.20433, 0.22584]

Mean RMSE: 0.22072


learning_rate: 0.1
Cross validation RMSE for NGBRegressor(early_stopping_rounds=100, learning_rate=0.1,
             random_state=RandomState(MT19937) at 0x2259022D440, verbose=False)

[0.21492, 0.24804, 0.22374, 0.20433, 0.21049]

Mea

## Подбираем количество иттераций

In [40]:
depth = 3
learning_rate = 0.05

for n_iter in [300,500,700,1000,1500]:
    print(f"n_iter: {n_iter}")
    score_model(
        NGBRegressor(
            Dist=Normal, 
            random_state=SEED, 
            verbose=False, 
            Base=DecisionTreeRegressor(criterion='friedman_mse', max_depth=depth),
            n_estimators=n_iter,
            learning_rate=learning_rate,
            early_stopping_rounds=100
        ), 
        X_train_forest, y_train
    )
    print()
    print()

n_iter: 300
Cross validation RMSE for NGBRegressor(early_stopping_rounds=100, learning_rate=0.05, n_estimators=300,
             random_state=RandomState(MT19937) at 0x2259022D440, verbose=False)

[0.21374, 0.2514, 0.21479, 0.19906, 0.22555]

Mean RMSE: 0.22091


n_iter: 500
Cross validation RMSE for NGBRegressor(early_stopping_rounds=100, learning_rate=0.05,
             random_state=RandomState(MT19937) at 0x2258F81F640, verbose=False)

[0.21456, 0.24927, 0.21487, 0.19925, 0.22489]

Mean RMSE: 0.22057


n_iter: 700
Cross validation RMSE for NGBRegressor(early_stopping_rounds=100, learning_rate=0.05, n_estimators=700,
             random_state=RandomState(MT19937) at 0x2258F81F640, verbose=False)

[0.21383, 0.24832, 0.21434, 0.19943, 0.22481]

Mean RMSE: 0.22015


n_iter: 1000
Cross validation RMSE for NGBRegressor(early_stopping_rounds=100, learning_rate=0.05, n_estimators=1000,
             random_state=RandomState(MT19937) at 0x2258F81F640, verbose=False)

[0.21431, 0.25087, 0.2146

In [41]:
depth = 3
learning_rate = 0.05

for n_iter in [600,700,800]:
    print(f"n_iter: {n_iter}")
    score_model(
        NGBRegressor(
            Dist=Normal, 
            random_state=SEED, 
            verbose=False, 
            Base=DecisionTreeRegressor(criterion='friedman_mse', max_depth=depth),
            n_estimators=n_iter,
            learning_rate=learning_rate,
            early_stopping_rounds=100
        ), 
        X_train_forest, y_train
    )
    print()
    print()

n_iter: 600
Cross validation RMSE for NGBRegressor(early_stopping_rounds=100, learning_rate=0.05, n_estimators=600,
             random_state=RandomState(MT19937) at 0x2259022D440, verbose=False)

[0.21389, 0.24806, 0.21779, 0.19844, 0.22501]

Mean RMSE: 0.22064


n_iter: 700
Cross validation RMSE for NGBRegressor(early_stopping_rounds=100, learning_rate=0.05, n_estimators=700,
             random_state=RandomState(MT19937) at 0x2258F81F640, verbose=False)

[0.21353, 0.24901, 0.21424, 0.19861, 0.22473]

Mean RMSE: 0.22002


n_iter: 800
Cross validation RMSE for NGBRegressor(early_stopping_rounds=100, learning_rate=0.05, n_estimators=800,
             random_state=RandomState(MT19937) at 0x2258F81F640, verbose=False)

[0.2147, 0.24695, 0.2149, 0.20026, 0.22545]

Mean RMSE: 0.22045




## col_samle

In [43]:
depth = 3
learning_rate = 0.05
n_iter = 700

for col_sample in [0.3, 0.5, 0.7]:
    print(f"col_sample: {col_sample}")
    score_model(
        NGBRegressor(
            Dist=Normal, 
            random_state=SEED, 
            verbose=False, 
            Base=DecisionTreeRegressor(criterion='friedman_mse', max_depth=depth),
            n_estimators=n_iter,
            learning_rate=learning_rate,
            early_stopping_rounds=100,
            col_sample=col_sample
        ), 
        X_train_forest, y_train
    )
    print()
    print()

col_sample: 0.3
Cross validation RMSE for NGBRegressor(col_sample=0.3, early_stopping_rounds=100, learning_rate=0.05,
             n_estimators=700,
             random_state=RandomState(MT19937) at 0x2259022DB40, verbose=False)

[0.20229, 0.24134, 0.21055, 0.20233, 0.21273]

Mean RMSE: 0.21385


col_sample: 0.5
Cross validation RMSE for NGBRegressor(col_sample=0.5, early_stopping_rounds=100, learning_rate=0.05,
             n_estimators=700,
             random_state=RandomState(MT19937) at 0x2259022DB40, verbose=False)

[0.21223, 0.24641, 0.21972, 0.1976, 0.21739]

Mean RMSE: 0.21867


col_sample: 0.7
Cross validation RMSE for NGBRegressor(col_sample=0.7, early_stopping_rounds=100, learning_rate=0.05,
             n_estimators=700,
             random_state=RandomState(MT19937) at 0x2259022DB40, verbose=False)

[0.20836, 0.2446, 0.22009, 0.20067, 0.22035]

Mean RMSE: 0.21881




In [82]:
depth = 3
learning_rate = 0.05
n_iter = 2000
col_sample = 0.3

modelOptimized = NGBRegressor(
            Dist=Normal, 
            random_state=SEED, 
            verbose=False, 
            Base=DecisionTreeRegressor(criterion='friedman_mse', max_depth=depth),
            n_estimators=n_iter,
            learning_rate=learning_rate,
            col_sample=col_sample
        )

In [83]:
modelOptimized.fit(X_train_forest, y_train)

NGBRegressor(col_sample=0.3, learning_rate=0.05, n_estimators=2000,
             random_state=RandomState(MT19937) at 0x2259022D240, verbose=False)

In [84]:
preds = modelOptimized.pred_param(X_test_forest)

In [88]:
preds_with_ci = get_ci(preds)
plot_data = np.round(np.expm1(np.c_[y_test, preds_with_ci]), -2)
result_ngboost = pd.DataFrame(plot_data, columns=["y", "pred", "lower95", "upper95"])
y_in_CI_rate = np.mean((result_ngboost.y > result_ngboost.lower95) & (result_ngboost.y < result_ngboost.upper95))
print(f"Значение y попадает в 95% доверительные интервал в {y_in_CI_rate*100:.0f}%")
print(f"RMSE = {mean_squared_error(modelOptimized.predict(X_test_forest), y_test)**0.5:2f}")

Значение y попадает в 95% доверительные интервал в 75%
RMSE = 0.196681


## А что покажет стандартная модель

In [92]:
modelDefault = NGBRegressor(
            Dist=Normal, 
            random_state=SEED, 
            verbose=False,
            n_estimators=700
        )

In [93]:
modelDefault.fit(X_train_forest, y_train)

NGBRegressor(n_estimators=700,
             random_state=RandomState(MT19937) at 0x22591CEA640, verbose=False)

In [95]:
preds = modelDefault.pred_param(X_test_forest)
preds_with_ci = get_ci(preds)
plot_data = np.round(np.expm1(np.c_[y_test, preds_with_ci]), -2)
result_ngboost = pd.DataFrame(plot_data, columns=["y", "pred", "lower95", "upper95"])
y_in_CI_rate = np.mean((result_ngboost.y > result_ngboost.lower95) & (result_ngboost.y < result_ngboost.upper95))
print(f"Значение y попадает в 95% доверительные интервал в {y_in_CI_rate*100:.0f}%")
print(f"RMSE = {mean_squared_error(modelDefault.predict(X_test_forest), y_test)**0.5:2f}")

Значение y попадает в 95% доверительные интервал в 90%
RMSE = 0.206115


> Получается мы улучшили предсказание за счет оптимизации модели, но ухудшили доверительные интервалы.
> А для нашего сервиса дов.интервалы важее. Поэтому возьмем стандартные параметры модели

In [96]:
preds = modelDefault.pred_param(X_test_forest)

In [104]:
np.mean((get_ci(preds)[:,1] < y_test) & ((get_ci(preds)[:,2] > y_test)))

-1.959963984540054
-1.959963984540054


0.8974358974358975